In [ ]:
import glob
import os

import pandas as pd
pd.set_option('display.max_colwidth', 255)

from dpp_helpline_qa.model_validation.model_validation import cal_em_score, calculate_semantic_similarity
from dpp_helpline_qa.modelling.question_answer import load_model_qa, answer_question
from dpp_helpline_qa.modelling.semantic_search import load_model_ss, context_ranking
from dpp_helpline_qa.preprocessing.preprocessing import process_docs

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
import torch
model_path = 'Rakib/roberta-base-on-cuad'
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

In [ ]:
import torch
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    squad_convert_examples_to_features
)

from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample
from transformers.data.metrics.squad_metrics import compute_predictions_logits

def run_prediction(question_texts, context_text, model_path):
    ### Setting hyperparameters
    max_seq_length = 512
    doc_stride = 100
    n_best_size = 1
    max_query_length = 100
    max_answer_length = 512
    do_lower_case = False
    null_score_diff_threshold = 0.0

    def to_list(tensor):
        return tensor.detach().cpu().tolist()

    config_class, model_class, tokenizer_class = (
        AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer)
    config = config_class.from_pretrained(model_path)
    tokenizer = tokenizer_class.from_pretrained(
        model_path, do_lower_case=True, use_fast=False)
    model = model_class.from_pretrained(model_path, config=config)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    processor = SquadV2Processor()
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs.to_tuple()]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    final_predictions = compute_predictions_logits(
        all_examples=examples,
        all_features=features,
        all_results=all_results,
        n_best_size=n_best_size,
        max_answer_length=max_answer_length,
        do_lower_case=do_lower_case,
        output_prediction_file=None,
        output_nbest_file=None,
        output_null_log_odds_file=None,
        verbose_logging=False,
        version_2_with_negative=True,
        null_score_diff_threshold=null_score_diff_threshold,
        tokenizer=tokenizer
    )

    return final_predictions

In [ ]:
ques = "Highlight the parts (if any) of this contract related to \"stock\" that should be reviewed? Details: My test counts have identified differences between the physical number of items in stock and the number recorded in the accounts. Does this mean that I can't place any reliance on the stock count as a control?"
ques1 = "My test counts have identified differences between the physical number of items in stock and the number recorded in the accounts. Does this mean that I can't place any reliance on the stock count as a control?"

context ='''My test counts have identified differences between the physical number of items in stock and the number recorded in the
accounts. Does this mean that I can't place any reliance on the stock count as a control?
No. If operating effectively, the client's stock count will detect and correct any errors in the stock balance recorded in the financial statements. Therefore a deviation is only identified in the control where KPMG's count differs from the client's final count and therefore an error has gone unnoticed.
What do I do if I identify an issue during an inventory count?
For each test count performed, there will be one of three outcomes:    
KPMG's and management's counts both agree to the inventory records: The inventory records are not misstated.
KPMG's and management's counts agree, but do not agree to the inventory records: In this case, management needs to update the inventory records. If the records are accurately updated, they are not misstated.
KPMG's and management's counts do not agree: See below.
'''
predictions = run_prediction(ques, context, model_path)

In [ ]:
predictions

In [ ]:
def get_answer_from_small_context(ques, context, model_path):
    # model & Tokenizer
    model = AutoModelForQuestionAnswering.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
    # prediction
    answer_list = []
    encoding = tokenizer.encode_plus(text=ques, text_pair=context)
    inputs = encoding['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(inputs)
    outputs = model(input_ids=torch.tensor([inputs]))
    # output
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer = tokenizer.convert_tokens_to_string(tokens[start_index:end_index+1]).strip()
    return answer

In [ ]:
#model_path = 'akdeniz27/roberta-large-cuad'
a = get_answer_from_small_context(ques1, context, model_path)
a

In [ ]:
model_path = 'akdeniz27/roberta-large-cuad'
a = get_answer_from_small_context(ques1, context, model_path)
a

In [ ]:
'''
We only get one opportunity to observe the inventory count and obtain the audit evidence we need. If we do not resolve issues on the day, they may be more difficult to resolve after the count.
If KPMG's and management's counts do not agree, we firstly recount and agree with management what the correct quantity should be. If we conclude that management had counted incorrectly, this may represent a control deviation in management's count control (if
we have not determined a tolerable difference)Í¾ this control deviation exists even if management updates the inventory records in agreement with our count result.
In all cases where a difference is identified between management's and KPMG's counts, it is important for the counter to discuss with the in-charge and manager whether further work is needed. This should be done BEFORE LEAVING SITE (or before finishing the virtual count) and ensures that senior members of the engagement team can consider the potential implications of any potential
deviations. This is true even if the individual count differences are not material, bearing in mind they may have a significant impact when we evaluate the control or the substantive sample (depending on the purpose of our count).
Additional information may be required around the nature and cause of the deficiency, or further test counts may be required.
Prior to leaving the count (or finishing the virtual count), it is important that we understand the process in place for any recounts and updates being made to the system. This may be later on the day of the count or potentially in the following few days after the count. The counter will need to ensure that they understand what follow up information they require, and request this prior to leaving (or finishing the virtual count). We also obtain, or request to be provided to us, copies of all count documentation (eg. count sheets) which demonstrate that all items due to be counted had been counted.
If your audit approach is a dual purpose approach where the substantive sample size has been set based on expected controls reliance you may need to reconsider RoMM and therefore sample sizes.
What do I need to bear in mind when I perform independent inventory counts (i.e. not counting at the same time as management's
counts)?
Remember that, even when we don't plan to test management's inventory count controls, ISA 501 requires us to attend management's inventory counting and perform certain specified procedures when inventory is material (ISA (UK) 501.4). This enables us to report back to management and those charged with governance if we have identified any concerns about the entity's inventory count procedures. Therefore, even when our substantive evidence will come from independent counts, we still need to attend at least one management count.   
Even where we don't plan to test the operating effectiveness of the entity's inventory count controls, counting at the same time as management may give us additional insight into management's count process, so can be preferable in that respect. We should, therefore, plan our substantive test counts alongside management's counts for at least a proportion of our count attendance, where possible and practical. If we have concerns with management's counts, then it may be appropriate to perform our own independent counts, and we should document our rationale for this approach.
Note that testing the entity's inventory count controls is often the most efficient and effective approach. When considering taking a fully substantive approach because the entity has not counted its inventory, or we anticipate their counts may be ineffective, first challenge management to remediate their processÍ¾ if management's process is deficient, our substantive count may identify misstatements that mean we need to ask management to investigate and correct the inventory records anyway, so it is better for them to avoid this by ensuring that their count process works before we move to our substantive testing.
Attendance at Physical Inventory Counting 
A1.  Management ordinarily establishes procedures under which inventory is physically  counted at least once a year to serve as a basis for the preparation of the financial  statements and, if applicable, to ascertain the reliability of the entityâ€™s perpetual  inventory system. 
A2.  Attendance at physical inventory counting involves: 
Inspecting the inventory to ascertain its existence and evaluate its condition, and  performing test counts; 
Observing compliance with managementâ€™s instructions and the performance of  procedures for recording and controlling the results of the physical inventory  count; and 
Obtaining audit evidence as to the reliability of managementâ€™s count procedures. 
These procedures may serve as test of controls or substantive procedures depending  on the auditorâ€™s risk assessment, planned approach and the specific procedures 
carried out. 
A3.  Matters relevant in planning attendance at physical inventory counting (or in designing  and performing audit procedures pursuant to paragraphs 4â€“8 of this ISA (UK)) include,  for example: 
The risks of material misstatement related to inventory. 
The nature of the internal control related to inventory. 
Whether adequate procedures are expected to be established and proper  instructions issued for physical inventory counting. 
The timing of physical inventory counting. 
Whether the entity maintains a perpetual inventory system. 
The locations at which inventory is held, including the materiality of the inventory  and the risks of material misstatement at different locations, in deciding at which  locations attendance is appropriate. ISA (UK) 600 (Revised June 2016)
 deals  with the involvement of other auditors and accordingly may be relevant if such  involvement is with regards to attendance of physical inventory counting at a  remote location. 
Whether the assistance of an auditorâ€™s expert is needed. ISA (UK) 620 (Revised  June 2016)
 deals with the use of an auditorâ€™s expert to assist the auditor to  obtain sufficient appropriate audit evidence. 
Evaluate Managementâ€™s Instructions and Procedures 
A4.  Matters relevant in evaluating managementâ€™s instructions and procedures for recording  and controlling the physical inventory counting include whether they address, for  example: 
The application of appropriate control activities, for example, collection of used  physical inventory count records, accounting for unused physical inventory count  records, and count and re-count procedures. 
The accurate identification of the stage of completion of work in progress, of slow  moving, obsolete or damaged items and of inventory owned by a third party, for  example, on consignment. 
The procedures used to estimate physical quantities, where applicable, such as  may be needed in estimating the physical quantity of a coal pile. 
Control over the movement of inventory between areas and the shipping and  receipt of inventory before and after the cutoff date. 
Observe the Performance of Managementâ€™s Count Procedures 
A5.  Observing the performance of managementâ€™s count procedures, for example those  relating to control over the movement of inventory before, during and after the count,  assists the auditor in obtaining audit evidence that managementâ€™s instructions and  count procedures are adequately designed and implemented. In addition, the auditor 
may obtain copies of cutoff information, such as details of the movement of inventory,  to assist the auditor in performing audit procedures over the accounting for such  movements at a later date. 
Inspect the Inventory 
A6.  Inspecting inventory when attending physical inventory counting assists the auditor in  ascertaining the existence of the inventory (though not necessarily its ownership), and  in identifying, for example, obsolete, damaged or ageing inventory. 
Perform Test Counts 
A7.  Performing test counts, for example by tracing items selected from managementâ€™s  count records to the physical inventory and tracing items selected from the physical  inventory to managementâ€™s count records, provides audit evidence about the  completeness and the accuracy of those records. 
A8.   In addition to recording the auditorâ€™s test counts, obtaining copies of managementâ€™s  completed physical inventory count records assists the auditor in performing  subsequent audit procedures to determine whether the entityâ€™s final inventory records  accurately reflect actual inventory count results.
Physical Inventory Counting Conducted Other than At the Date of the Financial Statements 
A9.   For practical reasons, the physical inventory counting may be conducted at a date, or  dates, other than the date of the financial statements. This may be done irrespective  of whether management determines inventory quantities by an annual physical  inventory counting or maintains a perpetual inventory system. In either case, the  effectiveness of the design, implementation and maintenance of controls over changes  in inventory determines whether the conduct of physical inventory counting at a date,  or dates, other than the date of the financial statements is appropriate for audit  purposes. ISA (UK) 330 (Revised July 2017) establishes requirements and provides  guidance on substantive procedures performed at an interim date.
A10.  Where a perpetual inventory system is maintained, management may perform physical  counts or other tests to ascertain the reliability of inventory quantity information  included in the entityâ€™s perpetual inventory records. In some cases, management or  the auditor may identify differences between the perpetual inventory records and  actual physical inventory quantities on hand; this may indicate that the controls over  changes in inventory are not operating effectively. 
A11.  Relevant matters for consideration when designing audit procedures to obtain audit  evidence about whether changes in inventory amounts between the count date, or  dates, and the final inventory records are properly recorded include: 
Whether the perpetual inventory records are properly adjusted. 
Reliability of the entityâ€™s perpetual inventory records. 
Reasons for significant differences between the information obtained during the  physical count and the perpetual inventory records.'''